In [1]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(
    role
)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

bucket = sess.default_bucket()  # Replace with your own bucket name if needed
print(bucket)
prefix = "fasttext/pretrained"  # Replace with the prefix under which you want to store the data if needed

arn:aws:iam::031342435657:role/service-role/AWSGlueServiceSageMakerNotebookRole-crb
sagemaker-us-east-1-031342435657


In [2]:
region_name = boto3.Session().region_name

In [3]:
container = sagemaker.amazon.amazon_estimator.image_uris.retrieve("blazingtext", region_name, "1")
print("Using SageMaker BlazingText container: {} ({})".format(container, region_name))

Using SageMaker BlazingText container: 811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:1 (us-east-1)


In [4]:
!wget -O model.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

--2021-09-01 09:48:40--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘model.bin’

model.bin           100%[===================>] 125.18M  21.0MB/s    in 6.6s    

2021-09-01 09:48:47 (18.9 MB/s) - ‘model.bin’ saved [131266198/131266198]



In [5]:
!tar -czvf langid.tar.gz model.bin
model_location = sess.upload_data("langid.tar.gz", bucket=bucket, key_prefix=prefix)
!rm langid.tar.gz model.bin

model.bin


In [6]:
lang_id = sagemaker.Model(
    image_uri=container, model_data=model_location, role=role, sagemaker_session=sess
)
lang_id.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

predictor = sagemaker.Predictor(
    endpoint_name=lang_id.endpoint_name,
    sagemaker_session=sess,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

-----!

In [9]:
sentences = [
    "hi which language is this?",
    "mon nom est Pierre",
    "hello man what is up",
    "नेपाली भाषा नेपालको सम्पर्क भाषा तथा भारत, भुटान र म्यानमारको केही भागमा मातृभाषाको रूपमा बोलिने भाषा हो.",
]
payload = {"instances": sentences}

In [10]:
predictions = predictor.predict(payload)
print(predictions)

[{'label': ['__label__en'], 'prob': [0.9948582053184509]}, {'label': ['__label__fr'], 'prob': [0.9984669089317322]}, {'label': ['__label__en'], 'prob': [0.90501469373703]}, {'label': ['__label__ne'], 'prob': [0.9716178178787231]}]
